In [1]:
pip install boto3 langchain psycopg2-binary pandas faiss-cpu

Note: you may need to restart the kernel to use updated packages.


I test if I can create a session and if it works: 

In [2]:
import boto3
import os

# Option 1: Use default credential provider chain
session = boto3.Session()

print(session)

Session(region_name='us-west-2')


All available LLM models in my region: 

In [3]:
import boto3

bedrock = boto3.client('bedrock', region_name='us-west-2')  # change to your region
models = bedrock.list_foundation_models()

for model in models['modelSummaries']:
    print(f"Model ID: {model['modelId']}")

Model ID: amazon.titan-tg1-large
Model ID: amazon.titan-embed-g1-text-02
Model ID: amazon.titan-text-lite-v1:0:4k
Model ID: amazon.titan-text-lite-v1
Model ID: amazon.titan-text-express-v1:0:8k
Model ID: amazon.titan-text-express-v1
Model ID: amazon.nova-pro-v1:0
Model ID: amazon.nova-lite-v1:0
Model ID: amazon.nova-micro-v1:0
Model ID: amazon.titan-embed-text-v1:2:8k
Model ID: amazon.titan-embed-text-v1
Model ID: amazon.titan-embed-text-v2:0
Model ID: amazon.titan-embed-image-v1:0
Model ID: amazon.titan-embed-image-v1
Model ID: amazon.titan-image-generator-v1:0
Model ID: amazon.titan-image-generator-v1
Model ID: amazon.titan-image-generator-v2:0
Model ID: amazon.rerank-v1:0
Model ID: stability.stable-diffusion-xl-v1:0
Model ID: stability.stable-diffusion-xl-v1
Model ID: stability.sd3-large-v1:0
Model ID: stability.sd3-5-large-v1:0
Model ID: stability.stable-image-core-v1:0
Model ID: stability.stable-image-core-v1:1
Model ID: stability.stable-image-ultra-v1:0
Model ID: stability.stable

Connection to Amazon Redshift:

In [5]:
import boto3

# Initialisation du client Redshift Data
client = boto3.client('redshift-data', region_name='us-west-2')

# Paramètres de connexion
database = 'dev'  # Remplace par le nom de ta base de données
workgroup_name = 'wz-solutions-redshift-workgroup'  # Remplace par ton workgroup

# Requête simple pour tester la connexion
sql_query = 'SELECT 1;'

try:
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name,  # Ne spécifie plus db_user ici
        Sql=sql_query
    )
    print("Connexion réussie !")
except Exception as e:
    print(f"Erreur de connexion : {e}")

Connexion réussie !


Test de faire une requete SQL à Amazon Redshift et voir si on peut avoir le résultat de la requete: 

In [ ]:
import boto3
import time

# Initialisation du client Redshift Data
client = boto3.client('redshift-data', region_name='us-west-2')

# Paramètres de connexion
database = 'dev'  # Remplace par le nom de ta base de données
workgroup_name = 'wz-solutions-redshift-workgroup'  # Remplace par ton workgroup

# Requête SQL à exécuter
sql_query = 'SELECT * FROM consoma LIMIT 10;'  # Remplace par ta table

try:
    # Exécuter la requête SQL
    response = client.execute_statement(
        Database=database,
        WorkgroupName=workgroup_name,  # Ne spécifie plus db_user ici
        Sql=sql_query
    )
    
    # Récupérer l'ID de l'exécution de la requête
    statement_id = response['Id']
    print(f"Requête envoyée, ID : {statement_id}")

    # Attendre que la requête soit terminée
    while True:
        status_response = client.describe_statement(Id=statement_id)
        status = status_response['Status']

        if status in ['FINISHED', 'FAILED', 'ABORTED']:
            break
        print("En attente des résultats...")
        time.sleep(2)  # Pause avant la prochaine vérification

    # Vérifier si la requête s'est bien exécutée
    if status == 'FINISHED':
        # Récupérer les résultats
        result_response = client.get_statement_result(Id=statement_id)
        records = result_response.get('Records', [])

        # Afficher les résultats
        if records:
            print("\nRésultats de la requête :")
            for row in records:
                print([col.get('stringValue', 'NULL') for col in row])  # Adaptation pour afficher chaque ligne
        else:
            print("Aucun résultat trouvé.")

    else:
        print(f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}")

except Exception as e:
    print(f"Erreur de connexion ou d'exécution : {e}")


Requête envoyée, ID : 979c0d74-e9e9-427c-97ce-50786006a8a5
En attente des résultats...

Résultats de la requête :
['GN_1180180102051605', 'GN|1180180102051605|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102342604', 'GN|1180180102342604|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102501103', 'GN|1180180102501103|22910|1|00000', '2022-10-10', 'NULL', '2022-03-27', '2022-09-02', 'NULL', 'NULL', 'NULL']
['GN_1180180102596604', 'GN|1180180102596604|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180102978210', 'GN|1180180102978210|22910|1|00000', '2022-10-10', 'NULL', '2022-03-27', '2022-09-02', 'NULL', 'NULL', 'NULL']
['GN_1180180103044102', 'GN|1180180103044102|22910|1|00000', '2022-10-10', 'NULL', '2022-03-28', '2022-09-03', 'NULL', 'NULL', 'NULL']
['GN_1180180103080601', 'GN|1180180103080601|22910|1|00000', '2022-10-10', '

Prompt 1: Analyse des tables et colonnes de notre base de données

In [5]:
import boto3
import json
import time

# ---- AWS CONFIGURATION ----
AWS_REGION = "us-west-2"
MODEL_ID = "mistral.mixtral-8x7b-instruct-v0:1"  # Utilisation de Mistral AI

# ---- REDSHIFT SERVERLESS CONFIGURATION ----
DATABASE = "dev"  # Remplace par ta base de données
WORKGROUP_NAME = "wz-solutions-redshift-workgroup"  # Remplace par ton workgroup

# ---- INITIALISATION DES CLIENTS ----
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
redshift_client = boto3.client("redshift-data", region_name=AWS_REGION)


# ---- FONCTION POUR RÉCUPÉRER LES TABLES REDSHIFT ----
def get_redshift_tables():
    """Récupère la liste des tables disponibles dans la base de données Redshift."""
    sql_query = "SELECT tablename FROM pg_catalog.pg_tables WHERE schemaname = 'public';"
    try:
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]
            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            time.sleep(2)

        if status == "FINISHED":
            result_response = redshift_client.get_statement_result(Id=statement_id)
            tables = [row[0].get('stringValue', 'NULL') for row in result_response.get("Records", [])]
            return tables if tables else "Aucune table trouvée."
        else:
            return f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        return f"Erreur de connexion ou d'exécution : {str(e)}"


# ---- FONCTION POUR RÉCUPÉRER LES COLONNES D'UNE TABLE ----
def get_table_columns(table_name):
    """Récupère les colonnes et leur description d'une table Redshift."""
    sql_query = f"""
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = '{table_name}';
    """
    try:
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]
            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            time.sleep(2)

        if status == "FINISHED":
            result_response = redshift_client.get_statement_result(Id=statement_id)
            columns = [
                f"{row[0].get('stringValue', 'NULL')} ({row[1].get('stringValue', 'NULL')})"
                for row in result_response.get("Records", [])
            ]
            return columns if columns else "Aucune colonne trouvée."
        else:
            return f"Erreur lors de l'exécution : {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        return f"Erreur de connexion ou d'exécution : {str(e)}"


# ---- FONCTION POUR ANALYSER LES TABLES AVEC L'AGENT ----
def agent_analyze_tables():
    """Analyse et décrit chaque table détectée dans Redshift."""
    tables = get_redshift_tables()
    if isinstance(tables, str):
        return tables

    all_tables_info = []
    for table in tables:
        columns = get_table_columns(table)
        all_tables_info.append(f"Table: {table}\nColonnes: {', '.join(columns)}")

    # Construire le prompt pour Mistral AI
    prompt = f"""
    Tu es un expert en bases de données. Analyse et décris chacune des tables trouvées dans Amazon Redshift.
    
    Voici les informations trouvées :
    {chr(10).join(all_tables_info)}

    Pour chaque table :
    1. Donne son rôle dans la base de données.
    2. Explique à quoi sert chaque colonne.
    
    Réponds de manière simple et concise, chaque description de colonne doit être de la même longueur et la plus courte et complète possible.
    """

    return analyze_with_mistral(prompt)


# ---- FONCTION POUR INTERAGIR AVEC MISTRAL AI ----
def analyze_with_mistral(prompt):
    """Envoie un prompt à Mistral AI via Amazon Bedrock."""
    request_body = {
        "prompt": prompt,
        "max_tokens": 800,
        "temperature": 0.3,
        "top_p": 0.9
    }

    response = bedrock.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps(request_body)
    )

    response_body = json.loads(response["body"].read())
    return response_body.get("outputs", [{}])[0].get("text", "")


# ---- EXÉCUTION ----
if __name__ == "__main__":
    response = agent_analyze_tables()
    print("\n📊 Analyse des Tables Redshift 📊\n")
    print(response) 
    rep_prompt1 = response 



📊 Analyse des Tables Redshift 📊


    Table: abonnements
    1. Le rôle de cette table est de stocker les informations relatives aux abonnements des clients.
    2. cle_abonnement (identifiant unique de l'abonnement), date_entree_local_abonnement (date d'entrée de l'abonnement), date_resiliation_abonnement (date de résiliation de l'abonnement), date_souscription_abonnement (date de souscription de l'abonnement).
    
    Table: consommations
    1. Le rôle de cette table est de stocker les informations relatives aux consommations d'eau des clients.
    2. annee_conso (année de la consommation), mois_conso (mois de la consommation), diametre_nominal (diamètre nominal de la canalisation), volume_mois (volume d'eau consommé pendant le mois), type_abaque (type d'abaque utilisé pour le calcul de la consommation), libelle_categorie_abonne (libellé de la catégorie de l'abonné), code_contrat (code du contrat), libelle_territoire (libellé du territoire), libelle_region (libellé de la région), 

In [8]:
import boto3
import json
import time


# ---- AWS CONFIGURATION ----
AWS_REGION = "us-west-2"
MODEL_ID = "mistral.mixtral-8x7b-instruct-v0:1"  # Utilisation de Mistral AI

# ---- REDSHIFT SERVERLESS CONFIGURATION ----
DATABASE = "dev"  
WORKGROUP_NAME = "wz-solutions-redshift-workgroup"  

# ---- INITIALISATION DES CLIENTS ----
bedrock = boto3.client("bedrock-runtime", region_name=AWS_REGION)
redshift_client = boto3.client("redshift-data", region_name=AWS_REGION)


# ---- FONCTION POUR RÉCUPÉRER UN ÉCHANTILLON D'UNE TABLE ----
def get_table_sample(table_name):
    """Récupère 1000 lignes d'une table Redshift pour analyse."""
    sql_query = f"SELECT * FROM {table_name} LIMIT 50;"
    
    try:
        print(f"🔄 Envoi de la requête à Redshift: {sql_query}")
        response = redshift_client.execute_statement(
            Database=DATABASE,
            WorkgroupName=WORKGROUP_NAME,
            Sql=sql_query
        )

        statement_id = response['Id']
        print(f"✅ Requête envoyée, ID: {statement_id}")

        # Timeout après 60 secondes
        start_time = time.time()
        while True:
            status_response = redshift_client.describe_statement(Id=statement_id)
            status = status_response["Status"]

            if status in ["FINISHED", "FAILED", "ABORTED"]:
                break
            
            # Vérification du timeout (60 secondes max)
            if time.time() - start_time > 60:
                print("⏳ Timeout dépassé (60s). Annulation de la requête.")
                return "Timeout: La requête a pris trop de temps."

            print("⏳ En attente des résultats...")
            time.sleep(3)  # Vérification toutes les 3 secondes

        if status == "FINISHED":
            print("✅ Requête terminée, récupération des résultats...")
            result_response = redshift_client.get_statement_result(Id=statement_id)
            records = [
                ", ".join([col.get('stringValue', 'NULL') for col in row])
                for row in result_response.get("Records", [])
            ]
            return records if records else "Aucune donnée trouvée."
        else:
            print(f"❌ Erreur d'exécution: {status}")
            return f"Erreur lors de l'exécution: {status_response.get('Error', 'Erreur inconnue')}"

    except Exception as e:
        print(f"❌ Erreur de connexion ou d'exécution : {str(e)}")
        return f"Erreur : {str(e)}"


# ---- FONCTION POUR DÉTECTER LES ANOMALIES ----
def agent_detect_anomalies(table_name):
    """Détecte les anomalies d'une table spécifique dans Redshift via Mistral AI."""
    table_sample = get_table_sample(table_name)
    
    if isinstance(table_sample, str):
        return table_sample  # Si erreur, on la renvoie directement

    formatted_sample = "\n".join(table_sample)

    # Construction du prompt
    prompt = f"""
    Tu es un expert en qualité des données.
    {rep_prompt1}
    
    Voici un échantillon de la table "{table_name}":
    {formatted_sample}
    
    Identifie les anomalies (valeurs nulles, doublons, erreurs de format, etc.).
    Génère une requête SQL pour afficher les lignes contenant des données incorrectes.

    Réponds en suivant ce format :
    - **Type d'anomalie** : [Catégorie de l'anomalie]
    - **Description** : [Brève explication]
    - **Requête SQL** : [Requête pour afficher les données erronées]
    """

    return analyze_with_mistral(prompt)


# ---- FONCTION POUR INTERAGIR AVEC MISTRAL AI ----
def analyze_with_mistral(prompt):
    """Envoie un prompt à Mistral AI via Amazon Bedrock."""
    request_body = {
        "prompt": prompt,
        "max_tokens": 400,
        "temperature": 0.3,
        "top_p": 0.9
    }

    try:
        print("🔄 Envoi du prompt à Mistral AI...")
        response = bedrock.invoke_model(
            modelId=MODEL_ID,
            body=json.dumps(request_body)
        )
        print("✅ Réponse reçue de Mistral.")

        response_body = json.loads(response["body"].read())
        return response_body.get("outputs", [{}])[0].get("text", "")

    except Exception as e:
        print(f"❌ Erreur d'interaction avec Mistral AI : {str(e)}")
        return f"Erreur : {str(e)}"


# ---- EXÉCUTION ----
if __name__ == "__main__":
    table_name = input("🔍 Entrez le nom de la table à analyser : ")
    response = agent_detect_anomalies(table_name)

    print("\n🔎 **Rapport d'Anomalies** 🔎\n")
    print(response)

🔄 Envoi de la requête à Redshift: SELECT * FROM factures LIMIT 50;
✅ Requête envoyée, ID: bfd6d9be-76fe-409f-bacd-f87c4f8839c9
⏳ En attente des résultats...
✅ Requête terminée, récupération des résultats...
🔄 Envoi du prompt à Mistral AI...
✅ Réponse reçue de Mistral.

🔎 **Rapport d'Anomalies** 🔎


    Anomalie 1 :
    - **Type d'anomalie** : Valeurs nulles
    - **Description** : Les champs "nb_jours_connus", "nb_factures_par_pds", "date_releve_index_facture", "date_releve_index_precedent_facture_composite" contiennent des valeurs nulles.
    - **Requête SQL** :
    ```
    SELECT * FROM factures
    WHERE nb_jours_connus IS NULL
    OR nb_factures_par_pds IS NULL
    OR date_releve_index_facture IS NULL
    OR date_releve_index_precedent_facture_composite IS NULL;
    ```

    Anomalie 2 :
    - **Type d'anomalie** : Doublons
    - **Description** : Les champs "num_fac_par_pds" et "cle_abonnement" contiennent des doublons.
    - **Requête SQL** :
    ```
    SELECT num_fac_par_pds, c